Preprocessing of the complaints 2020 NTA dataset

In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point # used to find the corresponding spatial zone
import sys
import os # to use functions defined in other scripts

# add scripts dir to path
# Add the relative path to the system path
edit_df_folder = os.path.abspath('../edit_df')
if edit_df_folder not in sys.path:
    sys.path.append(edit_df_folder)

# Import the editing scripts
import filter_rows
import filter_columns
import add_spatial_zone
import add_month

# add data subdirectories to path
core_folder =  os.path.abspath('../core_datasets')
if core_folder not in sys.path:
    sys.path.append(core_folder)
    
core_folder_complaints =  os.path.abspath('../core_datasets/complaints')
if core_folder_complaints not in sys.path:
    sys.path.append(core_folder_complaints)
    
scenarios_folder =  os.path.abspath('../scenarios')
if scenarios_folder not in sys.path:
    sys.path.append(scenarios_folder)
    
coordinates_maps_folder =  os.path.abspath('../coordinates_maps')
if coordinates_maps_folder not in sys.path:
    sys.path.append(coordinates_maps_folder)

Filter columns

In [20]:
core_df_path = os.path.join(core_folder_complaints,
                                   "NYPD_Complaint_Data_Historic_20241202.csv")

# load first df of variables to keep
var_v1_to_keep_df = pd.read_csv("complaints_var_to_keep.csv")

# filter columns
df = filter_columns.FilterColumns(df = pd.read_csv(core_df_path), var_df = var_v1_to_keep_df)


C:\Users\User\AppData\Local\Temp\ipykernel_23020\3668619359.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = filter_columns.FilterColumns(df = pd.read_csv(core_df_path), var_df = var_v1_to_keep_df)


Filter rows: keep only year 2020

In [21]:
df = filter_rows.FilterRowsContains(df = df,
                                    var_name = "RPT_DT",
                                    accepted_var_values = ["2020"])

Add MONTH variable

In [22]:
df = add_month.AddMONTH(df = df,
                        date_var_name= "RPT_DT",
                        date_format = '%m/%d/%Y')

Add NTA indicator variables

In [23]:

# Read the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(os.path.join(coordinates_maps_folder, "nta.geojson"))

df = add_spatial_zone.ConvertToGeodf(df,
                                     long = "Longitude",
                                     lat = "Latitude",
                                     crs = gdf.crs)
# actually join the two: by inclusion: df coordinates which are in polygons defined by gdf data
df = add_spatial_zone.SJoinWithinGeo(geodf_units = df,
                                     geodf2_polygons = gdf)

Second column filtering.
Remove latitude and longitude coordinates variables.
Remove complete date time variable.
(WARNING: we're using a different columns to keep file: complaints_v2)

In [24]:
# load first df of variables to keep
var_v2_to_keep_df = pd.read_csv("complaints_v2_var_to_keep.csv")

# filter columns
df = filter_columns.FilterColumns(df = df, var_df = var_v2_to_keep_df)

Look for missing and most likely values.

In [33]:
for column in df.columns:
    print(f"Unique values in column '{column}': { df[column].value_counts()}")

Unique values in column 'CRM_ATPT_CPTD_CD': CRM_ATPT_CPTD_CD
COMPLETED    407023
ATTEMPTED      6616
Name: count, dtype: int64
Unique values in column 'JURISDICTION_CODE': JURISDICTION_CODE
0     372249
2      30684
1       7224
97      1551
3       1041
88       252
72       214
14       150
4         87
11        79
15        26
87        21
13        18
12        15
9         13
6          6
85         5
7          4
Name: count, dtype: int64
Unique values in column 'KY_CD': KY_CD
341    82008
578    66799
344    43381
109    35738
351    35649
       ...  
102        2
234        2
357        1
676        1
571        1
Name: count, Length: 64, dtype: int64
Unique values in column 'LAW_CAT_CD': LAW_CAT_CD
MISDEMEANOR    210678
FELONY         135643
VIOLATION       67318
Name: count, dtype: int64
Unique values in column 'LOC_OF_OCCUR_DESC': LOC_OF_OCCUR_DESC
INSIDE         217057
FRONT OF       111733
UNKNOWN         66113
OPPOSITE OF     10031
REAR OF          8382
OUTSIDE         

First we uniform missing values to UNKNOWN

In [26]:
df.replace(['(null)'], 'UNKNOWN', inplace=True)

Second we remove non understandale age group values

In [32]:
df.replace(["2020", "2019", "-977", "-962", "-71", "-12", "-942", "1020", "-965", "1925", "-928",
            "-948", "-967", "-4", "-958", "943", "-968", "949", "-973", "-2", "932", "-31", "-938",
            "1016", "1014", "-60", "-1", "938", "950", "-963"],
           'UNKNOWN',
           inplace=True)

In [31]:
for column in df.columns:
    print(f"Unique values in column '{column}': { df[column].value_counts()}")

Unique values in column 'CRM_ATPT_CPTD_CD': CRM_ATPT_CPTD_CD
COMPLETED    407023
ATTEMPTED      6616
Name: count, dtype: int64
Unique values in column 'JURISDICTION_CODE': JURISDICTION_CODE
0     372249
2      30684
1       7224
97      1551
3       1041
88       252
72       214
14       150
4         87
11        79
15        26
87        21
13        18
12        15
9         13
6          6
85         5
7          4
Name: count, dtype: int64
Unique values in column 'KY_CD': KY_CD
341    82008
578    66799
344    43381
109    35738
351    35649
       ...  
102        2
234        2
357        1
676        1
571        1
Name: count, Length: 64, dtype: int64
Unique values in column 'LAW_CAT_CD': LAW_CAT_CD
MISDEMEANOR    210678
FELONY         135643
VIOLATION       67318
Name: count, dtype: int64
Unique values in column 'LOC_OF_OCCUR_DESC': LOC_OF_OCCUR_DESC
INSIDE         217057
FRONT OF       111733
UNKNOWN         66113
OPPOSITE OF     10031
REAR OF          8382
OUTSIDE         

Group my same column value

In [34]:
df = df.groupby(list(df.columns)).size().reset_index(name='count')

In [35]:
df.shape

(384541, 15)